In [ ]:
import pandas as pd
import plotly.express as px
from pathlib import Path

In [ ]:
# Lade CSV-Daten
df = pd.read_csv('h5p_content_types_analysis.csv')

# Entferne Prozentzeichen für numerische Verarbeitung
df['Prozent_num'] = df['Prozent'].str.rstrip('%').astype(float)

print(f"📊 Gesamt: {df['Anzahl'].sum()} H5P-Module")
print(f"📋 Verschiedene Content-Typen: {len(df)}\n")

## Vollständige Tabelle aller Content-Typen

In [ ]:
# Zeige vollständige Tabelle
display(df[['Content-Typ', 'Anzahl', 'Prozent']].style.background_gradient(
    subset=['Anzahl'], 
    cmap='YlOrRd'
))

## Top 15 Content-Typen

In [ ]:
# Top 15 für bessere Lesbarkeit
top_15 = df.head(15).copy()

# Kürze lange Namen für Visualisierung
top_15['Content-Typ_kurz'] = top_15['Content-Typ'].apply(
    lambda x: x[:50] + '...' if len(x) > 50 else x
)

display(top_15[['Content-Typ', 'Anzahl', 'Prozent']])

## Visualisierung: Top 15 Content-Typen

In [ ]:
# Balkendiagramm der Top 15
fig = px.bar(
    top_15,
    x='Anzahl',
    y='Content-Typ_kurz',
    orientation='h',
    title='Top 15 H5P Content-Typen (von 1262 Modulen)',
    labels={'Content-Typ_kurz': 'Content-Typ', 'Anzahl': 'Anzahl Module'},
    text='Prozent',
    color='Anzahl',
    color_continuous_scale='YlOrRd'
)

fig.update_layout(
    height=600,
    yaxis={'categoryorder': 'total ascending'},
    showlegend=False
)

fig.update_traces(textposition='outside')
fig.show()

## Kategorisierung nach H5P-Haupttyp

In [ ]:
# Extrahiere Haupttypen
def categorize_h5p_type(content_type):
    """Kategorisiert H5P Content-Typen in Hauptkategorien"""
    if 'interactiveVideo' in content_type:
        return 'Interactive Video'
    elif 'questions' in content_type or 'Question' in content_type:
        return 'Question Set / Quiz'
    elif 'H5P.content' in content_type:
        return 'Generic Content'
    elif 'dropzone' in content_type.lower() or 'drag' in content_type.lower():
        return 'Drag & Drop'
    elif 'presentation' in content_type.lower():
        return 'Presentation'
    elif 'timeline' in content_type.lower():
        return 'Timeline'
    elif 'hotspot' in content_type.lower():
        return 'Image Hotspot'
    elif 'branching' in content_type.lower():
        return 'Branching Scenario'
    elif 'accordion' in content_type.lower() or 'panels' in content_type.lower():
        return 'Accordion / Panels'
    elif 'cards' in content_type.lower() or 'flashcard' in content_type.lower():
        return 'Flashcards / Dialog Cards'
    elif 'words' in content_type.lower():
        return 'Word-based (Mark/Find)'
    else:
        return 'Andere'

df['Kategorie'] = df['Content-Typ'].apply(categorize_h5p_type)

# Gruppiere nach Kategorie
kategorie_stats = df.groupby('Kategorie').agg({
    'Anzahl': 'sum',
    'Content-Typ': 'count'
}).rename(columns={'Content-Typ': 'Anzahl_Typen'}).sort_values('Anzahl', ascending=False)

kategorie_stats['Prozent'] = (kategorie_stats['Anzahl'] / df['Anzahl'].sum() * 100).round(1).astype(str) + '%'

print("\n📦 H5P Content nach Hauptkategorien:\n")
display(kategorie_stats)

## Visualisierung: Verteilung nach Hauptkategorien

In [ ]:
# Pie Chart der Kategorien
fig_pie = px.pie(
    kategorie_stats.reset_index(),
    values='Anzahl',
    names='Kategorie',
    title='H5P Content-Verteilung nach Hauptkategorien',
    color_discrete_sequence=px.colors.sequential.RdBu
)

fig_pie.update_traces(textposition='inside', textinfo='percent+label')
fig_pie.show()

## Zusammenfassung

### Wichtigste Erkenntnisse:

1. **Interactive Video dominiert** mit 35% aller H5P-Module
2. **Question Sets / Quizzes** sind die zweithäufigste Kategorie (19%)
3. **Generic Content** (H5P.content) macht 14.6% aus
4. **Drag & Drop** Aktivitäten sind ebenfalls häufig vertreten
5. Es gibt eine **hohe Diversität** - 55 verschiedene Content-Typ-Varianten

### Implikationen für ETL-Pipeline:

- **Derzeit:** Nur Interactive Video wird extrahiert (nur Transkripte)
- **Problem:** 65% der H5P-Module werden ignoriert!
- **Handlungsbedarf:** Extraktion für Question Sets, Generic Content, Drag & Drop implementieren